# LSGAN

In [ ]:
import os, sys
import torch, torchvision
import torch.nn as nn

from torchvision import transforms
from torchvision.utils import save_image

sys.path.append(os.pardir)
from utils import *

print(torch.__version__)

### Settings

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

latent_size = 64
image_size = 28*28
hidden_size = 256
batch_size = 100

num_epochs = 200
learning_rate = 0.0002

In [ ]:
sample_dir = 'samples/LSGAN/FMNIST/'

if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [ ]:
MNIST_dataset = torchvision.datasets.MNIST(root='./data/MNIST/',
                                          train=True,
                                          transform=transform,
                                          download=True)

MNIST_loader = torch.utils.data.DataLoader(dataset=MNIST_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [ ]:
FMNIST_dataset = torchvision.datasets.FashionMNIST(root='./data/FMNIST/',
                                                  train=True,
                                                  transform=transform,
                                                  download=True)

FMNIST_loader = torch.utils.data.DataLoader(dataset=FMNIST_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

### Model

In [ ]:
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh()
)

D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1)
)

G = G.to(device)
D = D.to(device)

### Loss Function & Optimizer

In [ ]:
criterion = nn.MSELoss()

G_Optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=(0.5, 0.999))
D_Optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate, betas=(0.5, 0.999))

### Train

In [ ]:
G_Losses = []
D_Losses = []

Dx = []
DGz = []

total_step = len(FMNIST_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(FMNIST_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)
        
        #
        #  D
        #
        
        outputs = D(images)
        D_Loss_Real = criterion(outputs, real_labels)
        Score_Real = outputs
        
        z = torch.randn(batch_size, latent_size).to(device)
        images_fake = G(z)
        outputs = D(images_fake)
        D_Loss_Fake = criterion(outputs, fake_labels)
        Score_Fake = outputs
        
        D_Loss = D_Loss_Real + D_Loss_Fake
        
        reset_grad()
        D_Loss.backward()
        D_Optimizer.step()
        
        #
        #  G
        #
        
        z = torch.randn(batch_size, latent_size).to(device)
        images_fake = G(z)
        outputs = D(images_fake)
        
        G_Loss = criterion(outputs, real_labels)
        
        reset_grad()
        G_Loss.backward()
        G_Optimizer.step()
        
        if (i+1) % 200 == 0:
             print('Epoch [{}/{}], Step [{}/{}], D_Loss : {:.4f}, G_Loss : {:.4f}, D(x) : {:.2f}, D(G(z)) : {:.2f}'
                 .format(epoch+1, num_epochs, i+1, total_step, D_Loss.item(), G_Loss.item(), Score_Real.mean().item(), Score_Fake.mean().item()))
            
    if (epoch + 1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
        
    D_Losses.append(D_Loss.item())
    G_Losses.append(G_Loss.item())
    
    Dx.append(Score_Real.mean().item())
    DGz.append(Score_Fake.mean().item())
        
    images_fake = images_fake.reshape(images_fake.size(0), 1, 28, 28)
    save_image(denorm(images_fake), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
    
torch.save(G.state_dict(), 'G_LSGAN.ckpt')
torch.save(D.state_dict(), 'D_LSGAN.ckpt')

### Result

In [ ]:
drawLoss({'G_Loss':D_Losses, 'D_Loss':G_Losses})

In [ ]:
drawLoss({'Dx':Dx, 'DGz':DGz})